In [2]:
import cv2, numpy as np

In [14]:
## orb로 특징점 추출 >> 호모그래피로 변환행렬 생성 >> 원근변환

src = cv2.imread("image/apple_book.jpg", 0)
target = cv2.imread('image/apple.jpg', 0)

orb = cv2.ORB_create(
    nfeatures = 40000,
)

kp1, des1 = orb.detectAndCompute(src, None)
kp2, des2 = orb.detectAndCompute(target, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
matches = bf.match(des1, des2)
matches = sorted(matches, key = lambda x: x.distance)


src_pts = np.float32([kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)
target_pts = np.float32([kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)


mtrx, mask = cv2.findHomography(src_pts, target_pts, cv2.RANSAC, 5.0)

h, w = target.shape
pts = np.float32([[[0,0]],[[0, h-1]],[[w-1,h-1]],[[w-1,0]]])
dst = cv2.perspectiveTransform(pts,mtrx)

matrix = cv2.getPerspectiveTransform(dst,pts)
warp = cv2.warpPerspective(src, matrix, (w,h))

matchesMask = mask.ravel().tolist()
res = cv2.drawMatches(src,kp1,target,kp2,matches,None,
                      matchesMask = matchesMask,
                      flags = cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)





cv2.imshow('res', res)
cv2.imshow("warp", warp)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import cv2, numpy as np

img1 = cv2.imread('image/ir1.jpg',0)
img2 = cv2.imread('image/rgb2.jpg',0)
img2 = cv2.resize(img2, dsize = (0,0), fx = 0.5, fy = 0.5)


# knnMatch  ---①
##detector = cv2.xfeatures2d.SURF_create(hessianThreshold = 100, nOctaves = 3,extended = False, upright = False)
detector = cv2.xfeatures2d.SIFT_create()
##detector = cv2.ORB_create( nfeatures = 40000 )

kp1, desc1 = detector.detectAndCompute(img1, None)
kp2, desc2 = detector.detectAndCompute(img2, None)
matcher = cv2.BFMatcher(cv2.NORM_L1, crossCheck = True)
matches = matcher.match(desc1, desc2)
# 매칭 결과를 거리기준 오름차순으로 정렬 ---③
matches = sorted(matches, key=lambda x:x.distance)
# 모든 매칭점 그리기 ---④
res1 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, \
                    flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

# 매칭점으로 원근 변환 및 영역 표시 ---⑤
src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)



# RANSAC으로 변환 행렬 근사 계산 ---⑥
mtrx, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
h,w = img1.shape
pts = np.float32([ [[0,0]],[[0,h-1]],[[w-1,h-1]],[[w-1,0]] ])
dst = cv2.perspectiveTransform(pts,mtrx)

matrix = cv2.getPerspectiveTransform(dst,pts)
img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
warp = cv2.warpPerspective(img2, matrix, (w , h))



# 정상치 매칭만 그리기 ---⑦
matchesMask = mask.ravel().tolist()
res2 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, \
                    matchesMask = matchesMask,
                    flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)



# 모든 매칭점과 정상치 비율 ---⑧
accuracy=float(mask.sum()) / mask.size
print("accuracy: %d/%d(%.2f%%)"% (mask.sum(), mask.size, accuracy))

# 결과 출력                    
cv2.imshow('warp', warp)
cv2.imshow('Matching-All', res1)
cv2.imshow('Matching-Inlier ', res2)
cv2.waitKey()
cv2.destroyAllWindows()